In [29]:
import dai
import pandas as pd
from bigmodule import M, I

import sys
sys.path.append("/home/aiuser/work/GitHub/BigQuant_Resources_Collection/BigQuant_Research_Frameworks/General_Tools")
sys.path.append("/home/aiuser/work/GitHub/BigQuant_Resources_Collection/BigQuant_Research_Frameworks/Strategy_Tools")
from General_Tools import *
from Stock_Factor_Strategy_Tools import *

In [2]:
sd = get_date_str('2019-12-31')
ed = get_date_str('2026-02-13')

In [3]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    dividend_yield_ratio AS factor
FROM cn_stock_prefactors
QUALIFY c_pct_rank(total_market_cap) > 0.20
AND c_pct_rank(pe_ttm) < 0.40
AND pe_ttm > 0 
AND ps_ttm < 2.5
"""

In [4]:
sql_trade = get_trade_sql(sql_factor, number_security=10, position_allocation="equal", rebalance_period="month", stock_pool="000300.SH")

In [5]:
strategy_df = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
strategy_df

,date,instrument,score,score_rank,position
0,2019-12-31,600188.SH,0.145833,1.0,0.1
1,2019-12-31,600019.SH,0.087108,2.0,0.1
2,2019-12-31,600028.SH,0.074364,3.0,0.1
3,2019-12-31,000898.SZ,0.065672,4.0,0.1
4,2019-12-31,601006.SH,0.058465,5.0,0.1
...,...,...,...,...,...
735,2026-01-30,000157.SZ,0.057013,6.0,0.1
736,2026-01-30,601166.SH,0.056684,7.0,0.1
737,2026-01-30,601229.SH,0.056277,8.0,0.1
738,2026-01-30,000001.SZ,0.055217,9.0,0.1


In [30]:
def backtest_market_neutral_strategy(df, capital_base):
    
    def BigTrader_Initialize(context):
        from bigtrader.finance.commission import PerOrder
        context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
        context.data_date = dai.query("SELECT date FROM mldt_cn_stock_calendar_daily WHERE is_month_end_trade = 1").df()

    def BigTrader_Before_Trading(context, data):
        pass

    def BigTrader_Handle_Tick(context, tick):
        pass

    def BigTrader_Handle_Data(context, data):

        import math

        df_now = context.data_date[context.data_date["date"] == data.current_dt.strftime("%Y-%m-%d")]
        if len(df_now) == 0:
            return

        from bigtrader.constant import OrderType
        from bigtrader.constant import Direction


        
        index_future = "IF8888.CFE"
        position_short = context.get_account_position(index_future, direction=Direction.SHORT)
        print(position_short)

        if (position_short > 0):
            context.buy_close(index_future, position_short, price, order_type=OrderType.MARKET)
        
        cash = context.get_available_cash()
        price = data.current(index_future, "open")
        qty = math.floor(cash / price)
        context.sell_open(index_future, qty, price, order_type=OrderType.MARKET)

    def BigTrader_Handle_Trade(context, trade):
        pass

    def BigTrader_Handle_Order(context, order):
        pass

    def BigTrader_After_Trading(context, data):
        pass

    BigTrader = M.bigtrader.v53(
        
        data = df,
        
        start_date = """""",
        end_date   = """""",
        
        initialize           = BigTrader_Initialize,
        before_trading_start = BigTrader_Before_Trading,
        handle_tick          = BigTrader_Handle_Tick,
        handle_data          = BigTrader_Handle_Data,
        handle_trade         = BigTrader_Handle_Trade,
        handle_order         = BigTrader_Handle_Order,
        after_trading        = BigTrader_After_Trading,
        
        capital_base = capital_base + random.uniform(0, 10),
        frequency="""daily""",
        product_type="""期货""",
        rebalance_period_type="""交易日""",
        rebalance_period_days="""1""",
        rebalance_period_roll_forward=True,
        backtest_engine_mode="""标准模式""",
        before_start_days=0,
        volume_limit=1,
        order_price_field_buy="""open""",
        order_price_field_sell="""open""",
        benchmark="""沪深300指数""",
        
        plot_charts=True,
        debug=False,
        backtest_only=False,
        m_name="""BigTrader"""
    ) 

    return BigTrader.raw_perf.read()

In [27]:
backtest_market_neutral_strategy(strategy_df, capital_base=1000000)

[2026-02-17 18:28:56] [info     ] bigtrader.v53 开始运行 ..
[2026-02-17 18:28:56] [info     ] pybacktest run 2019-12-31 ~ 2026-01-30, , future, instruments=78
[2026-02-17 18:28:56] [info     ] bigtrader module V2.2.0
[2026-02-17 18:28:56] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-17 18:28:57] [info     ] pybacktest read history data done, run backtest ...
{}
FuturePosition(bktfut,IF8888.CFE,current_qty:(0, 0),avail_qty:(0, 0),last_price:0.0)


strategy strategy exception:Traceback (most recent call last):
  File "/opt/pyenv/versions/3.11.8/lib/python3.11/site-packages/bigtrader/strategy/engine.py", line 514, in _call_strategy_func
  File "/opt/pyenv/versions/3.11.8/lib/python3.11/site-packages/bigtrader/strategy/strategy_base.py", line 2426, in call_handle_data
  File "/tmp/ipykernel_497/3918462787.py", line 32, in BigTrader_Handle_Data
    if (position_short > 0):
        ^^^^^^^^^^^^^^^^^^
TypeError: '>' not supported between instances of 'FuturePosition' and 'int'



2026-02-17 18:28:57.228289 strategy strategy exception:Traceback (most recent call last):
  File "/opt/pyenv/versions/3.11.8/lib/python3.11/site-packages/bigtrader/strategy/engine.py", line 514, in _call_strategy_func
  File "/opt/pyenv/versions/3.11.8/lib/python3.11/site-packages/bigtrader/strategy/strategy_base.py", line 2426, in call_handle_data
  File "/tmp/ipykernel_497/3918462787.py", line 32, in BigTrader_Handle_Data
    if (position_short > 0):
        ^^^^^^^^^^^^^^^^^^
TypeError: '>' not supported between instances of 'FuturePosition' and 'int'
 
event engine process events exception err='>' not supported between instances of 'FuturePosition' and 'int'
Traceback (most recent call last):
  File "/opt/pyenv/versions/3.11.8/lib/python3.11/site-packages/bigtrader/event/event_engine.py", line 83, in process_events
  File "/opt/pyenv/versions/3.11.8/lib/python3.11/site-packages/bigtrader/event/event_engine.py", line 131, in _process_without_lock
  File "/opt/pyenv/versions/3.11.8/l

TypeError: '>' not supported between instances of 'FuturePosition' and 'int'